In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
conf = pyspark.SparkConf() \
            .setAppName("syl") \
            .setMaster("spark://master:7077") \
            .set("spark.blockManager.port", "10025") \
            .set("spark.driver.blockManager.port", "10026") \
            .set("spark.driver.port", "10027") \
            .set("spark.cores.max", "2")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/04 14:26:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [36]:
spark.stop()

In [4]:
from pyarrow import fs
import pyarrow as pa
import subprocess
import os

classpath = subprocess.Popen(["/opt/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='192.168.0.160', port=8020, user='hadoop')

2024-07-04 14:26:16,622 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
import datetime

today_news = datetime.datetime.now().strftime("%Y%m%d_mbc")

In [8]:
spark_df = spark.read.csv(f"hdfs:///P3T5/{today_news}.csv", 
               header=True, sep='|')

In [9]:
spark_df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Contents: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Organizer: string (nullable = true)



In [10]:
spark_df.limit(5).show()

+---------------------------------+---------------------------------+----------------+---------+
|                            Title|                         Contents|            Date|Organizer|
+---------------------------------+---------------------------------+----------------+---------+
|   "여당 의원 '꿀잠' 포착되자 ...| 여당 의원 꿀잠 포착되자 에효‥...|2024-07-04 12:17|      mbc|
|신임 방통위원장 후보에 이진숙 ...|신임 방통위원장 후보에 이진숙 ...|2024-07-04 12:05|      mbc|
|  '채상병 특검법' 무제한 토론‥...| 채상병 특검법 무제한 토론‥이 ...|2024-07-04 12:03|      mbc|
|   "민주, ""국민의힘, 필리버스...| 민주, 국민의힘, 필리버스터로 ...|2024-07-04 11:48|      mbc|
|  "국민의힘 당권 주자들, 총선 ...| 국민의힘 당권 주자들, 총선 책...|2024-07-04 11:39|      mbc|
+---------------------------------+---------------------------------+----------------+---------+



## 특수문자 제거, \n 제거

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col

first_row = spark_df.select("title").first()
#cleaned_title = regexp_replace(first_row["title"], "[^a-zA-Z0-9\s]", "")
clenad_title = spark_df.withColumn("cleaned_title", regexp_replace(col("title"), "[^a-zA-Z0-9\s]", ""))


In [35]:
cleaned_title.show(truncate=False)

TypeError: 'Column' object is not callable